# Custom Exceptions and Exception Chaining

**Chapter 6 - Learning Python, 5th Edition**

Building custom exception hierarchies for domain-specific error handling,
and using exception chaining to preserve error context.

## Defining Custom Exceptions

Custom exceptions should inherit from `Exception` (not `BaseException`) and
form a hierarchy that matches your domain:

In [ ]:
# Base exception for our application
class AppError(Exception):
    """Base exception for the application."""


class ValidationError(AppError):
    """Raised when input validation fails."""

    def __init__(self, field: str, message: str) -> None:
        self.field = field
        self.message = message
        super().__init__(f"{field}: {message}")


class NotFoundError(AppError):
    """Raised when a resource is not found."""

    def __init__(self, resource: str, identifier: str) -> None:
        self.resource = resource
        self.identifier = identifier
        super().__init__(f"{resource} '{identifier}' not found")


class AuthenticationError(AppError):
    """Raised when authentication fails."""


# Using custom exceptions
def validate_age(age: int) -> None:
    if age < 0 or age > 150:
        raise ValidationError("age", f"must be 0-150, got {age}")


try:
    validate_age(200)
except ValidationError as e:
    print(f"Caught: {e}")
    print(f"Field: {e.field}")
    print(f"Message: {e.message}")
    print(f"Is AppError? {isinstance(e, AppError)}")

## Exception Chaining with `raise ... from`

Exception chaining preserves the original error context when translating
low-level exceptions into domain exceptions:

In [ ]:
class ConfigError(AppError):
    """Raised when configuration is invalid."""


def load_config(raw: dict[str, str]) -> dict[str, int]:
    """Parse config values, chaining errors for context."""
    try:
        return {
            "port": int(raw["port"]),
            "timeout": int(raw["timeout"]),
        }
    except KeyError as e:
        raise ConfigError(f"Missing required config key: {e}") from e
    except ValueError as e:
        raise ConfigError(f"Invalid config value: {e}") from e


# Missing key
try:
    load_config({"port": "8080"})
except ConfigError as e:
    print(f"ConfigError: {e}")
    print(f"Caused by: {e.__cause__}")

print()

# Invalid value
try:
    load_config({"port": "abc", "timeout": "30"})
except ConfigError as e:
    print(f"ConfigError: {e}")
    print(f"Caused by: {type(e.__cause__).__name__}: {e.__cause__}")

## Suppressing Exception Context

Use `raise ... from None` to suppress the implicit exception chain
when the original error would be confusing to the caller:

In [ ]:
def get_user_name(user_id: int) -> str:
    """Look up user, hiding internal implementation details."""
    users = {1: "Alice", 2: "Bob"}
    try:
        return users[user_id]
    except KeyError:
        # Suppress the KeyError - caller doesn't need to know
        # about internal dict implementation
        raise NotFoundError("User", str(user_id)) from None


try:
    get_user_name(99)
except NotFoundError as e:
    print(f"Error: {e}")
    print(f"Cause suppressed: {e.__cause__}")
    print(f"Suppress context: {e.__suppress_context__}")

## Exception Groups (Python 3.11+)

Exception groups allow raising and handling multiple exceptions simultaneously,
useful for concurrent operations:

In [ ]:
def validate_user(name: str, age: int, email: str) -> None:
    """Validate all fields, collecting all errors."""
    errors: list[Exception] = []

    if not name.strip():
        errors.append(ValidationError("name", "cannot be empty"))
    if age < 0:
        errors.append(ValidationError("age", "must be non-negative"))
    if "@" not in email:
        errors.append(ValidationError("email", "must contain @"))

    if errors:
        raise ExceptionGroup("Validation failed", errors)


try:
    validate_user("", -5, "invalid")
except ExceptionGroup as eg:
    print(f"Group: {eg}")
    for i, exc in enumerate(eg.exceptions):
        print(f"  Error {i + 1}: {exc}")

## Practical Pattern: Error Accumulator

A common pattern for batch operations that should report all failures:

In [ ]:
from dataclasses import dataclass, field


@dataclass
class BatchResult:
    """Result of a batch operation with success/failure tracking."""

    succeeded: list[str] = field(default_factory=list)
    failed: list[tuple[str, Exception]] = field(default_factory=list)

    @property
    def all_succeeded(self) -> bool:
        return len(self.failed) == 0


def process_batch(items: list[str]) -> BatchResult:
    """Process items, collecting errors instead of failing fast."""
    result = BatchResult()

    for item in items:
        try:
            value = int(item)
            result.succeeded.append(f"{item} -> {value}")
        except ValueError as e:
            result.failed.append((item, e))

    return result


batch = process_batch(["42", "hello", "7", "world", "99"])
print(f"Succeeded: {batch.succeeded}")
print(f"Failed: {[(item, str(e)) for item, e in batch.failed]}")
print(f"All succeeded: {batch.all_succeeded}")